# Light Training: Self-Supervised Learning

**Mục tiêu:** học **biểu diễn** tốt mà **không dùng nhãn**; sau đó đánh giá bằng **linear probe** (đóng băng encoder, huấn luyện 1 classifier tuyến tính nhỏ).

Trong phần này, ta cài đặt **2 thuật toán**:
1. **SimCLR (NT-Xent Contrastive)** — dùng **cặp tăng cường** của cùng ảnh (positive) và negative là các ảnh còn lại trong batch.  
   - Biến thể: **Projection Head 2-layer** vs **3-layer**.
2. **BYOL (Bootstrap Your Own Latent)** — **không có negative**; dùng **target network** cập nhật bằng **EMA**.  
   - Biến thể: **EMA tĩnh** (m=0.99) vs **EMA theo lịch cosine**.


## (A) SimCLR — NT-Xent Contrastive

**Ý tưởng:** Tạo **hai view tăng cường** từ cùng ảnh (positive); những ảnh khác trong batch là **negative**. Tối đa hóa tương quan positive và tối thiểu hóa với negative bằng **NT-Xent**.

- **Kiến trúc:** `encoder` → `projection head (MLP)` → **chuẩn hoá**.  
- **Loss:** với batch B, ghép 2B embedding, xây **ma trận tương quan** 2B×2B, bỏ đường chéo, đặt **positive** ở vị trí tương ứng.  
- **Biến thể:**  
  - **Head-2L:** MLP 2 tầng (Linear→ReLU→Linear).  
  - **Head-3L:** MLP 3 tầng (Linear→ReLU→Linear→ReLU→Linear) — có thể biểu diễn tốt hơn nhưng tốn chút params/latency.

**Checkpoint:** `ssl_simclr_2l.pth`, `ssl_simclr_3l.pth` (best theo Val@1 từ linear probe định kỳ).  



## (B) BYOL — Bootstrap Your Own Latent (không negative)

**Ý tưởng:** Hai nhánh: **online** (encoder + projector + predictor) và **target** (encoder + projector).  
- **Loss:** cực đại hóa **cosine similarity** giữa **predictor(online(view1))** và **projector(target(view2))** (và ngược lại), với **stop-grad** ở nhánh target.  
- **Cập nhật target:** **EMA** từ online: \\( \theta_{t} \leftarrow m \theta_{t} + (1-m)\theta_{online} \\).  
- **Biến thể:**  
  - **EMA tĩnh**: m = 0.99.  
  - **EMA theo lịch cosine**: m tăng dần từ m0→m1 theo lịch cosine theo epoch.

**Checkpoint:** `ssl_byol_static.pth`, `ssl_byol_cosmo.pth` (best theo Val@1 từ linear probe định kỳ).  

